In [1]:
#### 对比实验 ####

In [2]:
# 导入环境
import sys
sys.path.append(r'./FederatedScope-master')
# 配置参数的定义
from federatedscope.core.configs.config import global_cfg
cfg = global_cfg.clone()

In [3]:
# # # ## fedpg算法参数
from xbl_codes.fed_proto_gan import FedPG_trainer
from xbl_codes.fed_proto_gan.FedPG_workers import FedPGServr, FedPGClient
cfg.fedpg.use = True
cfg.expname = 'FedPG'
cfg.federate.method = 'fedpg'
cfg.trainer.type = 'fedpg_trainer'
cfg.fedpg.vdata_per_class = 256
cfg.fedpg.v_batch_size = 64
# cfg.fedpg.weight_vdata = 1 #{0.1, 0.2, 0.5, 1)s
cfg.fedpg.weight_align = 20 #5
cfg.fedpg.upper_bound = 30
# cfg.fedpg.weight_decay = 0.95

# 保存模型的轮次
cfg.federate.server_save_round = [1, 100, 200, 300]
# cfg.federate.client_save_round = [1, 99, 101, 199, 201, 299, 301]
# cfg.federate.save_to = '../models/fedpg_lda0.1/' # 模型保存位置

In [4]:
# cfg.federate.method = 'fedavg'
# cfg.federate.server_save_round = [1, 100, 200, 300]
# cfg.federate.client_save_round = [1, 99, 101, 199, 201, 299, 301]
# cfg.federate.save_to = '../models/fedavg_lda0.1/' # 模型保存位置

In [5]:
# 模型参数
cfg.model.type = 'resnet18'
cfg.model.in_channels = 3
cfg.model.out_channels = 10

In [6]:
# 数据参数
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.cv.dataset  import cifar
from xbl_codes.utils import noniid_splitters

cfg.data.type = 'cifar10'
# cfg.data.transform = [['ToTensor'], ['Normalize', {'mean':[0.1307], 'std':[0.3081]}], ['Resize',{'size': [64, 64]}]] # femnist
cfg.data.root = './data'

cfg.data.splits = [0.85, 0.05, 0.1]

# 必须要加的超参数，否则acc=0.0(但是对于femnist来说不需要lda划分
cfg.data.splitter = 'lda'
cfg.data.splitter_args = [{'alpha':0.1}]

cfg.data.shuffle = True
cfg.data.batch_size = 64

In [7]:
# 训练参数
cfg.train.optimizer.lr = 0.1

cfg.train.optimizer.weight_decay = 0.0
cfg.grad.grad_clip = 5.0
cfg.early_stop.patience = 10

cfg.criterion.type = 'CrossEntropyLoss'

In [8]:
# 系统参数
cfg.wandb.use = True
# cfg.wandb.name_project = 'fedpg_config'
cfg.wandb.client_train_info = False

cfg.use_gpu = True
cfg.seed = 908

cfg.eval.freq = 1
cfg.eval.metrics = ['acc', 'f1']
cfg.eval.best_res_update_round_wise_key = 'test_acc'


cfg.federate.mode = 'standalone'
# cfg.federate.method = 'fedavg'
# cfg.federate.method = 'global'

cfg.federate.make_global_eval = True
cfg.federate.merge_test_data = True
cfg.federate.merge_val_data = True

cfg.federate.total_round_num = 100

cfg.train.batch_or_epoch = 'epoch'
cfg.train.local_update_steps = 5


cfg.federate.client_num = 10
cfg.federate.sample_client_num = 5

In [9]:
# 传参，记录
from federatedscope.core.auxiliaries.utils import setup_seed
from federatedscope.core.auxiliaries.logging import update_logger
setup_seed(cfg.seed)
update_logger(cfg)

2024-03-24 09:15:10,711 (logging:124)INFO: the current machine is at 10.96.3.179
2024-03-24 09:15:10,714 (logging:126)INFO: the current dir is C:\Users\xbl\exp_codes\FederatedScope-master\xbl_codes\fed_proto_gan
2024-03-24 09:15:10,716 (logging:127)INFO: the output dir is exp\FedPG\sub_exp_20240324091510


wandb: Currently logged in as: xuebl236. Use `wandb login --relogin` to force relogin


In [10]:
## 加载数据
data, modified_cfg = get_data(cfg.clone())
cfg.merge_from_other_cfg(modified_cfg)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
# # FL开工！
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls

server=get_server_cls(cfg)
client=get_client_cls(cfg)
Fed_runner = FedRunner(data=data,
                       server_class=server,
                       client_class=client,
                       config=cfg.clone())
Fed_runner.run()

KeyboardInterrupt: 

In [ ]:
# import numpy as np
# from torchvision import transforms
# import torchvision
# from torch.utils.data import DataLoader
# tfs = transforms.Compose([
#         transforms.Resize([64, 64]),
#         transforms.ToTensor()
#     ])

# v_dataset = torchvision.datasets.ImageFolder('data/v_femnist/', transform=tfs)
# v_dataloader = DataLoader(v_dataset, batch_size=64, shuffle=True)

# print(v_dataset)
# print(v_dataloader)
# # for x, y in v_dataloader:
# #     print(x.size())
# #     print(y.size())
# v_iter = iter(v_dataloader)
# batch_x, batch_y = next(v_iter)
# print(batch_x.size())
# print(batch_y)

In [ ]:
# server.show_generated_images()